In [1]:
import pandas as pd
import numpy as np
import math

import time

In [2]:
train = pd.read_csv('dataset/train_data.csv')
test = pd.read_csv('dataset/test_data.csv')

In [3]:
def baseline(train):
    mean = train["rating"].mean(skipna = True)
    users = train.pivot(index='movie_id', columns='user_id', values='rating')
    items = train.pivot(index='user_id', columns='movie_id', values='rating')
    nusers = train.values[:, 0].max()+1
    nitems = train.values[:, 1].max()+1
    bu = np.full(nusers, np.nan, dtype=float)
    bi = np.full(nitems, np.nan, dtype=float)

    for i in items.columns.values:
        bi[i] = np.nanmean(items[i] - mean)

    aux = bi[~np.isnan(bi)]
    for u in users.columns.values:
        bu[u] = np.nanmean(users[u] - mean - aux)

    bi = np.nan_to_num(bi)
    bu = np.nan_to_num(bu)
    model = { "bu" : bu, "bi": bi, "mean": mean }
    def predict(u, i):
        return model["mean"] + model["bu"][u] + model["bi"][i]

    model["pred"] = predict
    return model

In [4]:
def knn(train):
    ratings = train.pivot(index='movie_id', columns='user_id', values='rating')
    sim = ratings.corr(method='pearson')

    model = { "sim": sim, "ratings": ratings }
    def predict(u, i, k = 5):
        sim = model["sim"]
        ratings = model["ratings"]
        ratings_T = ratings.T
        if u not in sim or i not in ratings_T:
            return 0
        sim_users = sim[u][~sim[u].isna()].sort_values(ascending=False).index
        rated_users = ratings_T[i][ratings_T[i] > 0].index
        sim_k = np.intersect1d(sim_users, rated_users)
        top_k = [x for x in sim_users if x in sim_k][:k]
        sum_sim = 0
        dem = 0
        mean_u = ratings[u].mean(skipna = True)
        for v in top_k:
            dem += sim[u][v] * (ratings[v][i] - ratings[v].mean(skipna = True))
            sum_sim += sim[u][v]
        if sum_sim == 0:
            return 0
        return mean_u + (dem/sum_sim)

    model["pred"] = predict
    return model

In [5]:
# Função SVD Otimizado
def svdopt(train, k, lr = 0.05, reg = 0.02,miter = 10):
    train.columns = [0, 1, 2, 3]
    global_mean = train[2].mean(skipna = True)
    nusers = train[0].max()+1
    nitems = train[1].max()+1
    bu = np.full(nusers, 0, dtype=float)
    bi = np.full(nusers, 0, dtype=float)
    P = np.random.normal(loc = 0, scale = 0.1, size=(nusers, k))
    Q = np.random.normal(loc = 0, scale = 0.1, size=(nitems, k))
    error = list()
    for l in range(0, miter):
        sq_error = 0
        for j in train.index:
            u = train[0][j]
            i = train[1][j]
            r_ui = train[2][j]
            pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
            e_ui = r_ui - pred
            sq_error += e_ui**2
            bu[u] += lr * e_ui # - reg * bu[u]
            bi[i] += lr * e_ui # - reg * bi[i]
            for f in range(k):
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])
        error.append(math.sqrt(sq_error/len(train)))
    model = {  "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q, "error": error }
    def predict(u, i):
        return model["global_mean"] + model["bu"][u] + model["bi"][i] + np.dot(model["P"][u], model["Q"][i])

    model["pred"] = predict
    return model

In [6]:
def hibrid(train, k = 5, lr = 0.05, reg = 0.02,miter = 10):
    return { "baseline": baseline(train), "svdopt": svdopt(train, k, lr, reg, miter) }

In [7]:
def predict(model, u, i, k = 5):
    return (0.5 * model["baseline"]["pred"](u, i)) + (0.5 * model["svdopt"]["pred"](u, i))

In [8]:
# Avaliando modelo
def rmse(model, test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [9]:
def validation(model, data, train_split = 0.75,  k = 5, lr = 0.05, reg = 0.02,miter = 10):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train, k, lr, reg, miter)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [10]:
validation(hibrid, train, 0.75, 5, 0.05, 0.02)

Tempo de treinamento em segundos:  462.22674083709717
RMSE : 0.8788366568280844
Tempo de predição em segundos:  1.31135892868042


In [11]:
# def results(model, test):
#     return [predict(model, t[1], t[2]) for t in test]

# Iniciando contagem
# start_time = time.time()

# hibrid = hibrid(train)

# results = results(hibrid, test.values)

# Finalizando contagem
# print("Tempo de execucao em segundos: ", time.time() - start_time)

# results = pd.DataFrame({ 'rating': results })
# results.insert(0, 'id', results.index)
# results.to_csv('results/hibrid_results.csv', encoding='utf-8', index=False)

## Execução 1 treino 90% k = 5 KNN users(0.5) e Baseline(0.5)
Tempo de treinamento em segundos:  321.6816825866699 <br />
RMSE : 1.0354636821269407 <br />
Tempo de predição em segundos:  1476.614355802536 <br />

## Execução 2 treino 90% k = 5 SVD otimizado(0.5) e Baseline(0.5)
Tempo de treinamento em segundos:  381.6872208118439 <br />
RMSE : 0.8794344529299144 <br />
Tempo de predição em segundos:  0.48278355598449707 <br />

## Execução 3 treino 75% k = 5, lr = 0.05, reg = 0.02, miter = 10 SVD otimizado(0.5) e Baseline(0.5)
Tempo de treinamento em segundos:  284.07723593711853 <br />
RMSE : 0.8802939889152938 <br />
Tempo de predição em segundos:  1.3872888088226318 <br />

## Execução 3 treino 75% k = 5, lr = 0.07, reg = 0.02, miter = 10 SVD otimizado(0.3) e Baseline(0.7)
Tempo de treinamento em segundos:  573.8085193634033 <br />
RMSE : 0.8904080931526971 <br />
Tempo de predição em segundos:  2.690932512283325 <br />

## Execução 3 treino 75% k = 5, lr = 0.07, reg = 0.02, miter = 10 SVD otimizado(0.7) e Baseline(0.3)
Tempo de treinamento em segundos:  429.23968267440796 <br />
RMSE : 0.8908806318231459 <br />
Tempo de predição em segundos:  1.5721001625061035 <br />

## Execução 3 treino 75% k = 5, lr = 0.07, reg = 0.02, miter = 10 SVD otimizado(0.6) e Baseline(0.4)
Tempo de treinamento em segundos:  570.896012544632 <br />
RMSE : 0.8865988098253447 <br />
Tempo de predição em segundos:  1.2990763187408447 <br />

## Execução 3 treino 75% k = 5, lr = 0.05, reg = 0.02, miter = 10 SVD otimizado(0.6) e Baseline(0.4)
Tempo de treinamento em segundos:  424.18460178375244 <br />
RMSE : 0.8832916993054506 <br />
Tempo de predição em segundos:  2.84799861907959 <br />

## Execução 3 treino 75% k = 5, lr = 0.05, reg = 0.02, miter = 10 SVD otimizado(0.5) e Baseline(0.5)
Tempo de treinamento em segundos:  462.22674083709717 <br />
RMSE : 0.8788366568280844 <br />
Tempo de predição em segundos:  1.31135892868042 <br />